In [2]:
import findspark
findspark.init("/Users/itn.gunit.mittal/spark")
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.regression import RandomForestRegressor

In [6]:
spark = SparkSession.builder.appName("rf_exmaple").getOrCreate()

22/04/30 18:01:20 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:67)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:483)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnection.waitForCommands(ClientServerConnection.ja

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.NoClassDefFoundError: Could not initialize class org.apache.spark.storage.StorageUtils$
	at org.apache.spark.storage.BlockManagerMasterEndpoint.<init>(BlockManagerMasterEndpoint.scala:110)
	at org.apache.spark.SparkEnv$.$anonfun$create$9(SparkEnv.scala:348)
	at org.apache.spark.SparkEnv$.registerOrLookupEndpoint$1(SparkEnv.scala:287)
	at org.apache.spark.SparkEnv$.create(SparkEnv.scala:336)
	at org.apache.spark.SparkEnv$.createDriverEnv(SparkEnv.scala:191)
	at org.apache.spark.SparkContext.createSparkEnv(SparkContext.scala:277)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:460)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:67)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
	at 

In [ ]:
data = spark.read.csv("data/dummy_data.csv", inferSchema=True, header=True)

In [ ]:
data.printSchema()

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
data.columns

In [17]:
import pyspark.sql.functions as F
import random
# data_modified = data.select("*", (F.col("fare_amount") - F.col("total_amount")).alias("weather"))
# data_modified.columns
data_modified = data.withColumn("weather", F.round(F.rand() * 10))

In [18]:
from datetime import datetime
from pyspark.sql.types import DoubleType
def getTripDuration(datetime_start, datetime_end):
    try:
        result = (datetime.strptime(datetime_end, "%Y-%m-%d %H:%M:%S") - datetime.strptime(datetime_start, "%Y-%m-%d %H:%M:%S")).total_seconds()
        return result
    except Exception:
        return 0

tripDurationFunction = F.udf(getTripDuration, DoubleType())
# datetime_object = datetime.strptime("2019-12-18 15:52:30", "%Y-%m-%d %H:%M:%S")
# datetime_object2 = datetime.strptime("2019-12-18 15:54:39", "%Y-%m-%d %H:%M:%S")
# print(f"This is datetime: {(datetime_object2 - datetime_object).total_seconds()}")
data_modified = data_modified.withColumn("trip_duration", tripDurationFunction("lpep_pickup_datetime", "lpep_dropoff_datetime"))
data_modified.show(3)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+-------+-------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|weather|trip_duration|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+-------+-------------+
|       2| 2019-12-18 15:52:30|  2019-12-18 15:54:39|                 N|         1|         264| 

In [19]:
data_modified.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge',
 'weather',
 'trip_duration']

In [22]:
assembler = VectorAssembler(inputCols=['PULocationID','DOLocationID','passenger_count','trip_distance','fare_amount','extra','mta_tax','tip_amount','tolls_amount','improvement_surcharge', 'total_amount','payment_type','trip_type','congestion_surcharge','weather','trip_duration'], outputCol="features")


In [24]:
output = assembler.transform(data_modified)
output.head(1)

[Row(VendorID=2, lpep_pickup_datetime='2019-12-18 15:52:30', lpep_dropoff_datetime='2019-12-18 15:54:39', store_and_fwd_flag='N', RatecodeID=1, PULocationID=264, DOLocationID=264, passenger_count=5, trip_distance=0.0, fare_amount=3.5, extra=0.5, mta_tax=0.5, tip_amount=0.01, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=4.81, payment_type=1, trip_type=1, congestion_surcharge=0.0, weather=2.0, trip_duration=129.0, features=DenseVector([264.0, 264.0, 5.0, 0.0, 3.5, 0.5, 0.5, 0.01, 0.0, 0.3, 4.81, 1.0, 1.0, 0.0, 2.0, 129.0]))]

In [25]:
final_data = output.select("features", "total_amount")

In [27]:
final_data.show()

+--------------------+------------+
|            features|total_amount|
+--------------------+------------+
|[264.0,264.0,5.0,...|        4.81|
|[66.0,65.0,2.0,1....|       24.36|
|[181.0,228.0,1.0,...|       15.34|
|[129.0,263.0,2.0,...|       25.05|
|[210.0,150.0,1.0,...|        11.3|
|[35.0,39.0,1.0,3....|        14.8|
|[25.0,61.0,1.0,2....|        12.3|
|[225.0,89.0,1.0,4...|        21.8|
|[129.0,129.0,1.0,...|         6.8|
|[129.0,83.0,1.0,0...|         6.8|
|[82.0,173.0,1.0,1...|        10.8|
|[74.0,69.0,1.0,3....|        15.3|
|[74.0,41.0,1.0,1....|         7.8|
|[41.0,127.0,1.0,5...|        20.3|
|[7.0,260.0,1.0,1....|        10.8|
|[7.0,7.0,1.0,1.42...|         8.3|
|[7.0,133.0,1.0,15...|       53.16|
|[134.0,28.0,1.0,1...|         7.8|
|[89.0,39.0,1.0,2....|        11.3|
|[66.0,65.0,3.0,1....|         7.8|
+--------------------+------------+
only showing top 20 rows



In [28]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [29]:
train_data.describe().show()

+-------+------------------+
|summary|      total_amount|
+-------+------------------+
|  count|                67|
|   mean|15.569701492537295|
| stddev| 9.920033750270859|
|    min|               4.8|
|    max|             53.16|
+-------+------------------+



In [30]:
test_data.describe().show()

+-------+------------------+
|summary|      total_amount|
+-------+------------------+
|  count|                31|
|   mean|17.497419354838712|
| stddev|15.890901373163604|
|    min|               4.8|
|    max|              78.1|
+-------+------------------+



In [31]:
rf = RandomForestRegressor(featuresCol="features", labelCol="total_amount")
rf_model = rf.fit(train_data)


In [ ]:
rfparamGrid = (ParamGridBuilder()
             #.addGrid(rf.maxDepth, [2, 5, 10, 20, 30])
               .addGrid(rf.maxDepth, [2, 5, 10])
             #.addGrid(rf.maxBins, [10, 20, 40, 80, 100])
               .addGrid(rf.maxBins, [5, 10, 20])
             #.addGrid(rf.numTrees, [5, 20, 50, 100, 500])
               .addGrid(rf.numTrees, [5, 20, 50])
             .build())

In [ ]:
rfevaluator = RegressionEvaluator(predictionCol="prediction", labelCol="total_amount", metricName="rmse")

In [ ]:
# Create 5-fold CrossValidator
rfcv = CrossValidator(estimator = rf,
                      estimatorParamMaps = rfparamGrid,
                      evaluator = rfevaluator,
                      numFolds = 5)

In [ ]:
rfcvModel = rfcv.fit(train_data)
print(rfcvModel)
rfpredictions = rfcvModel.transform(test_data)
print('RMSE:', rfevaluator.evaluate(rfpredictions))